In [1]:
import pandas as pd
import openpyxl
import os
from datetime import datetime

In [2]:
# use os.path.join.  Use r literal

PATH = os.path.abspath(r"C:\Users\Angel\Downloads\group_stats_detailed_150537_sxrslhqfqv.xls")

In [3]:
df = pd.read_excel(PATH, header =1)

In [4]:
df.head(5)
# has properly loaded

,Provider,Facility,Shift,Date,Day,Time,Work Hrs,Sched Hrs,Work w/Ratio,Sched w/Ratio
0,Franklin Alconcel,SRDHSM,day,03/29/2023,Wed,6a - 2p,8.0,8.0,8.0,8.0
1,Franklin Alconcel,SRDHSM,night,03/10/2023,Fri,10p - 6a,8.0,8.0,8.0,8.0
2,Franklin Alconcel,SRDHSM,night,03/09/2023,Thu,10p - 6a,8.0,8.0,8.0,8.0
3,Franklin Alconcel,SRSC,Float,03/20/2023,Mon,12p - 7p,7.0,7.0,7.0,7.0
4,Caleb Anderson,SRDHSM,swing,03/18/2023,Sat,2p - 10p,8.0,8.0,8.0,8.0


In [5]:
# add f_name, l_name, end time.
name_data = df['Provider'].str.split(" ", n=1, expand=True)
df['f_name'] = name_data[0]
df['l_name'] = name_data[1]
time_data = df['Time'].str.split(" - ", n=1, expand=True)
df['end_time'] = time_data[1] + 'm'

In [6]:
df.head(5)

,Provider,Facility,Shift,Date,Day,Time,Work Hrs,Sched Hrs,Work w/Ratio,Sched w/Ratio,f_name,l_name,end_time
0,Franklin Alconcel,SRDHSM,day,03/29/2023,Wed,6a - 2p,8.0,8.0,8.0,8.0,Franklin,Alconcel,2pm
1,Franklin Alconcel,SRDHSM,night,03/10/2023,Fri,10p - 6a,8.0,8.0,8.0,8.0,Franklin,Alconcel,6am
2,Franklin Alconcel,SRDHSM,night,03/09/2023,Thu,10p - 6a,8.0,8.0,8.0,8.0,Franklin,Alconcel,6am
3,Franklin Alconcel,SRSC,Float,03/20/2023,Mon,12p - 7p,7.0,7.0,7.0,7.0,Franklin,Alconcel,7pm
4,Caleb Anderson,SRDHSM,swing,03/18/2023,Sat,2p - 10p,8.0,8.0,8.0,8.0,Caleb,Anderson,10pm


In [7]:
# Converts Date to a datetime format
df['Date'] = pd.to_datetime(df['Date'])
df['end_time'] = pd.to_datetime(df['end_time'], format='%I%p')

In [8]:
df.dtypes

Provider                 object
Facility                 object
Shift                    object
Date             datetime64[ns]
Day                      object
Time                     object
Work Hrs                float64
Sched Hrs               float64
Work w/Ratio            float64
Sched w/Ratio           float64
f_name                   object
l_name                   object
end_time         datetime64[ns]
dtype: object

In [9]:
df = df[df['Shift'].notna()]

In [10]:
df = df.reset_index(drop=True)

In [11]:
df.shape

(313, 13)

In [12]:
PATH_request = os.path.abspath(
    r"C:\Users\Angel\Downloads\schedule_period_request_graph_wadmin.xlsx"
)

In [13]:
df_r = pd.read_excel(PATH_request, header =1)
df_r = df_r.drop(df_r.tail(4).index)
df_r = df_r.drop(columns=['On','Off','Time Of'])

In [14]:
df_r.tail()

,User,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,Comments
46,Sabrina Taylor,,,,,,,,,,...,,,,,,,,,,Please limit my night shifts to 2-3 this month.
47,Joseph Tran,,,,,,,,,,...,,,,,,,,,,Prefer blocked shifts up to 7 days. Would like...
48,Kyle Vanstone,,,,,,,,,,...,,,,,,,,,,"all overnights North Las Vegas, sorry for limi..."
49,Irena Vitkovitsky,,,,,,,,,,...,,,,,,,,,,NaN
50,Harold Woo,,,,,,,,,,...,,,,,,,,,,Please consider that i was scheduled for 7 ove...


In [15]:
for i, row in df.iterrows():
    name = row['Provider']
    day = row['Date'].day
    end_time = row['end_time']
    print(end_time.time())
    late_times = ['20:00:00', '21:00:00', '22:00:00', '23:00:00', '01:00:00', '02:00:00', '06:00:00']

    # if provider is in df_r
    if name not in df_r.User.tolist():

        # If Provider in df doesn't exist in df_r, create row in df with 'User':name
        new_row = pd.DataFrame({'User': name}, index=[-1])
        df_r2 = pd.concat([df_r.loc[:], new_row]).reset_index(drop=True)
        df_r = df_r2.copy()

    # find index of provider in df
    provider_index = df_r.User[df_r.User == name ].index.tolist()[0]

    # write onto the cell with an indicator
    indicator = 'w'
    if str(end_time.time()) in late_times:
        print(f"end_time is {type(end_time)}")
        indicator += ' til ' + str(end_time.time())[:5]
    df_r.at[provider_index, str(day)] = indicator

        



14:00:00
06:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
06:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
19:00:00
22:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
21:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
21:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
06:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
14:00:00
14:00:00
14:00:00
14:00:00
14:00:00
14:00:00
01:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
21:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
18:00:00
22:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
14:00:00
14:00:00
21:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
17:00:00
17:00:00
17:00:00
21:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
21:00:00
end_time is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
21:00:00
e

In [16]:
df_r.to_excel('scratch.xlsx')

In [17]:
from openpyxl import load_workbook


wb = load_workbook(PATH_request, data_only = True)
wb.save(filename='test.xslx')
sh = wb['Request Graph']

In [18]:
provider_hex_d = dict(zip(
    [row[0] for row in sh.iter_rows(min_row=3, max_row=(sh.max_row - 4), max_col=(1), values_only=True)], 
    
    
    [[cell.fill.start_color.index  for cell in row] for row in sh.iter_rows(min_col=2, max_col=(sh.max_column-4), min_row=3, max_row=(sh.max_row - 4))]
    ))




provider_hex_d


{'Franklin Alconcel': [9,
  '00000000',
  '00000000',
  10,
  10,
  '00000000',
  11,
  11,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  '00000000',
  '00000000',
  '00000000',
  '00000000',
  10,
  10,
  '00000000',
  '00000000',
  9,
  9,
  9],
 'Lance Allgower': [9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  '00000000',
  '00000000',
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9],
 'Jefferson Bracey': [9,
  11,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  11,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  11,
  9,
  9,
  9,
  9],
 'Kenneth Chang': [9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  10,
  10,
  '00000000',
  '00000000',
  '00000000',
  '00000000',
  '00000000'],
 'Christopher Cheng': [9,
  '00000000',
  '00000000',
  10,
  10,
  '00000000',
  9,
  9,
  '00000000',
  9,
  10,
  10,
  '00000000',
  9,
  9,


In [ ]:
PATH_request_processed1 = os.path.abspath(
    r"C:\Users\Angel\anaconda3\envs\scratch.xlsx"
)
# for every User in dictionary, for every list, if either 10, 11, or 9, fill.


wb_r = load_workbook(PATH_request_processed1, data_only=True)
wb_r.save(filename = 'processed2.xslx')
sh_r = wb_r['Sheet1']


In [ ]:
from openpyxl.styles import PatternFill

color_d = {'10': '00C0C0C0' ,'11':'00808080', '12':'00b200','9': 'ff8b94', '00000000': '00FFFFFF' }

for row in sh_r.iter_rows(min_col=2, max_col=(sh_r.max_column - 1), min_row=2, max_row=(sh_r.max_row)):
    key = row[0].value

    try:

        for i in range(len(provider_hex_d[key])):
            #fill cell  and move to next cell
            cellc_to_fill = row[i + 1].coordinate

            code = str(provider_hex_d[key][i])
            sh_r[cellc_to_fill].fill = PatternFill(fgColor=color_d[code], fill_type = "solid")

    except KeyError:
        pass

            


In [ ]:
wb_r.save(filename = 'processed2.xlsx')